## MultiHead Self Attention

In [2]:
import torch
import torch.nn as nn

In [3]:
# causal attention class

class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

## Multihead attention class

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, number_of_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([
            
            CausalAttention(d_in, d_out, context_length,dropout) for _ in range(number_of_heads)
            
        ])
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [10]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89],
   [0.55, 0.87, 0.66],
   [0.57, 0.85, 0.64],
   [0.22, 0.58, 0.33], 
   [0.77, 0.25, 0.10], 
   [0.05, 0.80, 0.55]] 
)

In [34]:
torch.manual_seed(123)

batch = torch.stack((inputs, inputs), dim=0)

context_length = batch.shape[-2]

d_out, d_in = 2, 3

multihead_attention = MultiHeadAttention(d_in, d_out, context_length, 0.0, 2, False)

mha_output = multihead_attention(batch)

In [30]:
# mha_output

In [35]:
B, N, D = mha_output.shape

print(f"Batch size: {B}, Number of tokens: {N}, Embedding dimension: {D}")

Batch size: 2, Number of tokens: 6, Embedding dimension: 4


## Replicating the original paper attention

In [43]:
NO_ATTENTION_HEADS = 8
D_EMBEDDING = 512
D_KEY = D_QUERY = D_VALUE = D_EMBEDDING // NO_ATTENTION_HEADS
BATCH_SIZE = 64

In [52]:
inputs_paper = torch.rand(BATCH_SIZE, 100, D_EMBEDDING)

multihead_attention_output = MultiHeadAttention(D_EMBEDDING, D_KEY, 100, 0.0, NO_ATTENTION_HEADS, False)(inputs_paper)


In [53]:
inputs_paper.shape

torch.Size([64, 100, 512])

In [54]:
multihead_attention_output.shape

torch.Size([64, 100, 512])